## Agile Hardware Design
***
# Formal Verification

## Kevin Laeufer
### laeufer@berkeley.edu

- Berkeley PhD student
- maintainer of the [chiseltest](https://github.com/ucb-bar/chiseltest) library

**Note**: make sure you have the [z3 SMT solver](https://github.com/Z3Prover/z3) installed to be able to execute the examples on your machine. It should be available through your favorite package manager: _brew_, _dnf_, _apt_ etc.

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Winter22/)

## Plan for Today

* boolean satisfiability and SMT
* verifying combinatorial circuits
* verifying sequential circuits
* applied examples

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.formal._
import chiseltest.RawTester._

# SMT Solver Boilerplate

A convenience function to make our SMT solving examples easier to read.

In [ ]:
import firrtl.backends.experimental.smt._
import chiseltest.formal.backends.smt._
/** helper function to easily check SMT formulas and return all assignments */
object solve {
  private val verbose: Boolean = true

  def apply(expr: Seq[BVExpr]): Map[String, BigInt] = apply(expr, false)
  def apply(expr: Seq[BVExpr], quiet: Boolean): Map[String, BigInt] =
    apply(BVAnd(expr.toList), quiet)
  def apply(expr: BVExpr): Map[String, BigInt] = apply(expr, false)
  def apply(expr: BVExpr, quiet: Boolean): Map[String, BigInt] = {
    require(expr.width == 1, s"Expression needs to be boolean, not ${expr.width}-bits wide.")
    val symbols = findSymbols(expr).distinct
    val solver = Z3SMTLib.createContext()
    solver.setLogic("ALL")
    symbols.foreach(s => solver.runCommand(DeclareFunction(s, Seq())))
    solver.assert(expr)
    val r = solver.check()
    r.toString match { // small hack because the result is package private
      case "IsSat" =>
        val values = symbols.map(s => s.name -> solver.getValue(s).get)
        val vStr = values.map{ case (n,v) => s"$n = $v" }.mkString(", ")
        solver.close()
        if(verbose && !quiet) println(s"$expr is sat: $vStr")
        values.toMap
      case "IsUnSat" =>
        solver.close()
        if(verbose && !quiet) println(s"$expr is unsat")
        Map()
    }
  }

  private def findSymbols(e: SMTExpr): Seq[BVSymbol] = e match {
    case s: BVSymbol    => Seq(s)
    case _: ArraySymbol => ???
    case other => other.children.flatMap(findSymbols)
  }
}

// define some symbols we can use later
val b0 = BVSymbol("b0", 1)
val b1 = BVSymbol("b1", 1)
val a = BVSymbol("a", 32)
val b = BVSymbol("b", 32)
object and { def apply(a: BVExpr, b: BVExpr): BVExpr = BVAnd(a, b) }
object or { def apply(a: BVExpr, b: BVExpr): BVExpr = BVOr(a, b) }
object add { def apply(a: BVExpr, b: BVExpr): BVExpr = BVOp(Op.Add, a, b) }
object sub { def apply(a: BVExpr, b: BVExpr): BVExpr = BVOp(Op.Sub, a, b) }
object mul { def apply(a: BVExpr, b: BVExpr): BVExpr = BVOp(Op.Mul, a, b) }
object leftShift { def apply(a: BVExpr, b: BVExpr): BVExpr = BVOp(Op.ShiftLeft, a, b) }
object equal { def apply(a: BVExpr, b: BVExpr): BVExpr = BVEqual(a, b) }
object gt { def apply(a: BVExpr, b: BVExpr): BVExpr = BVComparison(Compare.Greater, a, b, false) }
object gte { def apply(a: BVExpr, b: BVExpr): BVExpr = BVComparison(Compare.GreaterEqual, a, b, false) }
object ite { def apply(c: BVExpr, a: BVExpr, b: BVExpr): BVExpr = BVIte(c, a, b) }
object not { def apply(a: BVExpr): BVExpr = BVNot(a) }
object lit { def apply(value: BigInt, width: Int = 32): BVExpr = BVLiteral(value, width) }
object failAfter {
  def apply(n: Int): Unit = {
    require(n > 0)
    val failCount = RegInit(n.U)
    failCount := failCount - 1.U
    assert(failCount =/= 0.U, s"failure triggered after $n cycles")
  }
}

# Boolean Satisfiability

- given a boolean formula: $ F(x_1, x_2, ..., x_n) $
- can $F$ evaluate to true?
- _or_ does the exist an assignment to the variables $x_1 ... x_n$ such that $F$ evaluates to true? `(sat)`
- _or_ does $F$ for all possible assignments to $x_1 .... x_n$ evaluate to false? `(unsat)`
- our `solve` determines satisfiability for us


[more information](https://ptolemy.berkeley.edu/projects/embedded/eecsx44/fall2011/lectures/SATSolving.pdf)

In [ ]:
solve(and(b0, not(b0)))
// solve(and(b0, b1))
// solve(or(b0, b1))

# Boolean Satisfiability
- can be solved with a truth table
- $F_0(b_0, b_1) = b_0 \land \neg b_0$
- $F_1(b_0, b_1) = b_0 \land b_1$


| $b_0$  | $F_0$ |
|--------|-------|
|  $0$   |  $0$  |
|  $1$   |  $0$  |



| $b_0$  | $b_1$ | $F_1$ |
|--------|-------|-------|
|  $0$   |  $0$  |  $0$  |
|  $0$   |  $1$  |  $0$  |
|  $1$   |  $0$  |  $0$  |
|  $1$   |  $1$  |  $1$  |

# Boolean Satisfiability
- can be solved with a truth table
- truth table size grows exponentially with number of variables: $\mathcal{O}(2^n)$
- first NP-complete problem (Cook, 1971)
- but a lot of general cases can be solved quickly!



# SMT
- extends SAT solving with _theories_ beyond boolean formulas
- **Bit Vectors** model fixed bit width integers
- **Arrays** model storing and loading from a memory
- both are important when trying to reason about C programs or digital circuits

In [ ]:
// a + a == 8
solve(equal(add(a, a), lit(8)))
// a + a == 9
// solve(equal(add(a, a), lit(9)))
// c * 4 == c << a
val c = 145
// solve(equal(mul(lit(c), lit(4)), leftShift(lit(c), a)))
// c * 5 == c << a
// solve(equal(mul(lit(c), lit(5)), leftShift(lit(c), a)))

# Verifying Combinatorial Circuits
- we call circuits without state elements (registers or memories) _combinatorial_
- assertions on these modules can be directly mapped to a SMT formula

```.scala
val a = IO(Input(UInt(32.W)))
val b = a + 1.U
assert(b > a)
```

In [ ]:
// does there exist an assignment to a such that the assertion _fails_
solve(and(
    equal(b, add(a, lit(1))), // define b
    not(gt(b, a))             // can the assertion be violated?
))
// what is going on?
// println("0" + (4294967295L).toBinaryString)
// println((4294967295L + 1).toBinaryString)

# Verifying Combinatorial Circuits
- now with chisel!

In [ ]:
class AddOne extends Module {
  val a = IO(Input(UInt(32.W)))
  val b = a + 1.U
  assert(b > a, "%b > %b", a, b)
}

verify(new AddOne, Seq(BoundedCheck(1)))

# Application of Combinatorial Circuit Verification
- gray code is used for counters that need to cross clock domains
- important properties:
 - can convert from/to binary without loss of information

In [ ]:
/** Checks that when we go from binary -> gray -> binary the result is always the same as the input. */
class GrayCodeIdentityCheck(width: Int) extends Module {
  val in = IO(Input(UInt(width.W)))
  val gray = BinaryToGray(in)
  val out = GrayToBinary(gray)
  assert(in === out, "%b -> %b -> %b", in, gray, out)
}
verify(new GrayCodeIdentityCheck(64), Seq(BoundedCheck(1)))

# Application of Combinatorial Circuit Verification
- gray code is used for counters that need to cross clock domains
- important properties:
 - can convert from/to binary without loss of information
 - when input is incremented, only one bit will flip

In [ ]:
/** Checks that if we increment the binary number, the gray code equivalent only changes by one bit. */
class GrayCodeHammingCheck(width: Int) extends Module {
  val a = IO(Input(UInt(width.W)))
  val b = a + 1.U
  val aGray = BinaryToGray(a)
  val bGray = BinaryToGray(b)
  val hamming = PopCount(aGray ^ bGray)
  assert(hamming === 1.U, "%b ^ %b = %b", aGray, bGray, hamming)
}
verify(new GrayCodeHammingCheck(64), Seq(BoundedCheck(1)))

# Combinatorial Circuit Verification
- need to write an assertion to define property that we want to check
- SMT solver can help us prove that for all possible inputs, the assertion cannot be violated
- with the Chisel interface, we choose `BoundedCheck(1)` for a full proof

# Verifying Sequential Circuits
- most circuits have state (registers or memories) which evolves over time

In [ ]:
class MyCounter extends Module {
    val en = IO(Input(Bool()))
    val count = RegInit(0.U(32.W))
    when(en && count === 22.U) { count := 0.U }
    when(en && count =/= 22.U) { count := count + 1.U }
    assert(count =/= 10.U)

    printf(p"count = ${count}, en = ${en}\n")
}

verify(new MyCounter, Seq(BoundedCheck(24)))

# Verifying Sequential Circuits
- transition system abstraction
<img src="transition-system.png" alt="transition system" style="width:55%;" align="center"/>

- $s_0$: `count = 0, en = 0`
- $s_1$: `count = 0, en = 1`
- $s_2$: `count = 1, en = 0`
- $s_e$: `count = 10, en = ?` (10 instead of 500 for now)
- can we reach $s_e$?

In [ ]:
case class State(counter: BVSymbol, en: BVSymbol)
def makeState(num: Int) = State(BVSymbol(s"counter_$num", 32), BVSymbol(s"en_$num", 1))
def I(s: State) = equal(s.counter, lit(0))
def T(s0: State, s1: State): BVExpr = {
  val nextCounter = ite(s0.en, ite(equal(s0.counter, lit(22)), lit(0), add(s0.counter, lit(1))), s0.counter)
  // enable is an input and thus left unconstrained
  equal(s1.counter, nextCounter)
}
def P(s: State) = not(equal(s.counter, lit(10))) // 10 instead of 500 for now
val (s0, s1) = (makeState(0), makeState(1))


# Transition System Questions
- what happened to our clock?
- what happened to our reset?

# Verifying Sequential Circuits

## Bounded model checking

- symbolic states: $s_i$, initial state: $I$, next state: $T$, assertion: $P$
- $I(s_0) \land T(s_0, s_1) \land ... \land T(s_{k-1}, s_k) \land ( \lor_{i=0}^{k} \neg P(s_i) )$

In [ ]:
// manual example
// check if the initial state violates the assertion
solve(Seq(I(s0), not(P(s0))))
// check if state 1 can violate the assertion
solve(Seq(I(s0), T(s0, s1), not(P(s1))))

// val s2 = makeState(2)
// solve(Seq(I(s0), T(s0, s1), T(s1, s2), not(P(s2))))

# Bounded Model Checking

In [ ]:
// implement a bmc routine
def bmc(k: Int): Unit = {
  var prev = makeState(0)
  var cond = List(I(prev))
  (0 to k).foreach { j =>
    val r = solve(cond :+ not(P(prev)), quiet=true)
    if(r.nonEmpty) {
      println(s"found bug after $j steps")
      println(r.toSeq.sorted.mkString("\n"))
      return
    }
    // add another state
    val state = makeState(j + 1)
    cond = cond :+ T(prev, state)
    prev = state
  }
}
bmc(5)

# Bounded Model Checking
step 0: $counter \in \{0\}$
<img src="ind0.png" alt="define induction state space" style="width:55%;" align="center"/>


# Bounded Model Checking
step 1: $counter \in \{0, 1\}$
<img src="bmc0.png" alt="define induction state space" style="width:55%;" align="center"/>


# Bounded Model Checking
step 2: $counter \in \{0, 1, 2\}$
<img src="bmc1.png" alt="define induction state space" style="width:55%;" align="center"/>


# Applications: Queues

In [ ]:
class QueueIO(bitWidth: Int) extends Bundle {
  val enq = Flipped(Decoupled(UInt(bitWidth.W)))
  val deq = Decoupled(UInt(bitWidth.W))
}
// from our queue lecture
class MyQueueV6(val numEntries: Int, bitWidth: Int, pipe: Boolean=true) extends Module {
  val io = IO(new QueueIO(bitWidth))
  require(numEntries > 1)
  //     require(isPow2(numEntries))    // no longer needed
  val entries = Mem(numEntries, UInt(bitWidth.W))
  val enqIndex = Counter(numEntries)
  val deqIndex = Counter(numEntries)
  val maybeFull = RegInit(false.B)
  val indicesEqual = enqIndex.value === deqIndex.value
  val empty = indicesEqual && !maybeFull
  val full = indicesEqual && maybeFull
  if (pipe)
    io.enq.ready := !full || io.deq.ready
  else
    io.enq.ready := !full
  io.deq.valid := !empty
  io.deq.bits := entries(deqIndex.value)
  val fixed = false
  if(fixed) {
    when (io.deq.fire =/= io.enq.fire) {
      maybeFull := io.enq.fire
    }
  } else {
    when (indicesEqual && io.deq.fire =/= io.enq.fire) {
      maybeFull := !maybeFull
     }
  }
  when (io.deq.fire) {
    deqIndex.inc()
  }
  when (io.enq.fire) {
    entries(enqIndex.value) := io.enq.bits
    enqIndex.inc()
  }
  // printf(p"deq.valid=${io.deq.valid}, enq.ready=${io.enq.ready}, maybeFull=${maybeFull}, enqIndex=${enqIndex.value}, deqIndex=${deqIndex.value}, enq.fire=${io.enq.fire}, deq.fire=${io.deq.fire}\n")
}


# Applications: Queues
- data integrity property:
  - when data enters the Queue it should exit again in the right order

# Queue Trace

In [ ]:
class QueueTrace extends Module {
    val dut = Module(new MyQueueV6(numEntries = 3, bitWidth = 8, pipe=false))
    val io = IO(new QueueIO(8)) ; io <> dut.io
    
    // count elements
    val elementCount = RegInit(0.U(log2Ceil(dut.numEntries + 1).W))
    val nextElementCount = elementCount + io.enq.fire - io.deq.fire 
    elementCount := nextElementCount
    
    
    // trace internal state
    val cycle = RegInit(0.U(8.W)) ; cycle := cycle + 1.U
    printf(p"${cycle} ----------------------------------\n")
    printf(p"count: ${elementCount} -> ${nextElementCount}\n")
    when(io.enq.fire) { printf(p"[${io.enq.bits}]") }
    when(io.enq.fire || io.deq.fire) { printf(" --> ") }
    when(io.deq.fire) { printf(p"[${io.deq.bits}]") }
    when(io.enq.fire || io.deq.fire) { printf("\n") }
    
    // failAfter(3)
}
verify(new QueueTrace, Seq(BoundedCheck(6)))

# Queue Check 1
- pick first element that enters the queue and check that it is correctly dequeued

In [ ]:
class QueueCheck01 extends QueueTrace {
    val isActive = RegInit(false.B) // are we tracking a packet?
    val packetValue = Reg(UInt(8.W))
    // start tracking when first packet enters
    when(!isActive && io.enq.fire) {
        isActive := true.B
        packetValue := io.enq.bits
        printf(p"tracking: ${io.enq.bits} @ ${elementCount}\n")
    } 
    when(isActive && io.deq.fire) {
        isActive := false.B
        assert(io.deq.bits === packetValue, "%d =/= %d", io.deq.bits, packetValue)
        when(io.deq.bits === packetValue) { printf(p"OK (${packetValue})\n") }
    }    
}
verify(new QueueCheck01, Seq(BoundedCheck(3)))

# Queue Check 2
- make sure we can track a second element

In [ ]:
class QueueCheck02 extends QueueTrace {
    val isActive = RegInit(false.B) // are we tracking a packet?
    val packetValue = Reg(UInt(8.W))
    val packetCount = Reg(chiselTypeOf(elementCount))
    // start tracking when first packet enters
    when(!isActive && io.enq.fire) {
        isActive := true.B; packetValue := io.enq.bits
        packetCount := nextElementCount
        printf(p"tracking: ${io.enq.bits} @ ${elementCount}\n")
    } 
    when(isActive && io.deq.fire) {
        packetCount := packetCount - 1.U
        when(packetCount === 1.U) {
            isActive := false.B
            assert(io.deq.bits === packetValue, "%d =/= %d", io.deq.bits, packetValue)
            when(io.deq.bits === packetValue) { printf(p"OK (${packetValue})\n") }
        }
    }    
}
verify(new QueueCheck02, Seq(BoundedCheck(10)))

# Queue Check 3
- pick **any** element that enters the queue and check that it is correctly dequeued

In [ ]:
class QueueCheck03 extends QueueTrace {
    val en = IO(Input(Bool())) // <-- add enable!
    val isActive = RegInit(false.B) // are we tracking a packet?
    val packetValue = Reg(UInt(8.W))
    val packetCount = Reg(chiselTypeOf(elementCount))
    // start tracking when first packet enters
    when(!isActive && en && io.enq.fire) {
        isActive := true.B; packetValue := io.enq.bits ; packetCount := nextElementCount
        printf(p"tracking: ${io.enq.bits} @ ${elementCount}\n")
    } 
    when(isActive && io.deq.fire) {
        packetCount := packetCount - 1.U
        when(packetCount === 1.U) {
            isActive := false.B
            assert(io.deq.bits === packetValue, "%d =/= %d", io.deq.bits, packetValue)
            when(io.deq.bits === packetValue) { printf(p"OK (${packetValue})\n") }
        }
    }    
}
verify(new QueueCheck03, Seq(BoundedCheck(10)))

# Formal Verification with Chisel
- formal verification can exhaustively verify a circuit
- checker needs to be written in Chisel (not software!)
- checker can be smaller if we find a good property to check (but can be hard to find the right one!)
- can find interesting corner case bugs
- [more examples](https://github.com/ekiwi/chisel-verification-examples) [to try out](https://github.com/ucb-bar/chiseltest/tree/master/src/test/scala/chiseltest/formal/examples)
- feel free to [get in touch](mailto:laeufer@eecs.berkeley.edu) if you have any questions
- [chiseltest library](https://github.com/ucb-bar/chiseltest)
- [chisel community chat](https://gitter.im/freechipsproject/chisel3)

# Bonus: temporal assertions
- allows us to specify properties involving multiple cycles
- an example form the [AXI4 specification](http://www.gstitt.ece.ufl.edu/courses/fall15/eel4720_5721/labs/refs/AXI4_specification.pdf):
<img src="axi.png" alt="axi4 property" style="width:55%;" align="center"/>


In [ ]:
class AxiCheck extends Module {
    val io = IO(Flipped(Irrevocable(UInt(8.W))))
    val ready = IO(Input(Bool())) ; io.ready := ready
    when(past(io.valid && !io.ready)) {
        assert(io.valid, "valid may not be deasserted without a ready")
        assert(stable(io.bits), "bits may not change until a transaction occurs (%d -> %d)", past(io.bits), io.bits)
    }
}

# Bonus: temporal assertions

In [ ]:
test(new AxiCheck) { dut =>
    dut.clock.step()
    
    dut.io.valid.poke(true.B)
    dut.io.bits.poke(123.U)
    dut.io.ready.expect(false.B)
    dut.clock.step()
    
    // transaction happens
    dut.ready.poke(true.B)
    dut.clock.step()
    
    // now we are allowed to change the bits
    dut.io.bits.poke(234.U)
    dut.clock.step()
    
    // no more transaction
    dut.ready.poke(false.B)
    dut.clock.step()
    
    // we are not allowed to change the data here
    // dut.io.bits.poke(123.U)
    dut.clock.step()
    
    // we are not allowed to revoke the valid
    dut.io.valid.poke(false.B)
    dut.clock.step()
}

# Bonus: temporal assertions
- SystemVerilog (SVA) and VHDL (PSL) support even more complicated temporal assertions
- [PSL assertion examples](https://github.com/tmeissner/psl_with_ghdl)
- [SystemVerilog Assertion examples](https://github.com/SymbioticEDA/sva-demos)

# Bonus: k-induction
- we can prove the example `Counter` correct by induction
- [original example on slide 99](https://zipcpu.com/tutorial/class-verilog.pdf )

In [ ]:
def P(s: State) = not(equal(s.counter, lit(500))) // now 500

# Review: Bounded Model Checking
step 0: $counter \in \{0\}$
<img src="ind0.png" alt="define induction state space" style="width:55%;" align="center"/>


# Review: Bounded Model Checking
step 1: $counter \in \{0, 1\}$
<img src="bmc0.png" alt="define induction state space" style="width:55%;" align="center"/>


# Review:  Bounded Model Checking
step 2: $counter \in \{0, 1, 2\}$
<img src="bmc1.png" alt="define induction state space" style="width:55%;" align="center"/>


# Bonus: k-induction
- we can prove the example `Counter` correct by induction
- [original example on slide 99](https://zipcpu.com/tutorial/class-verilog.pdf )
- base case: $I(s_0) \land \neg P(s_0)$: `counter === 0 && !(counter =/= 500)`
- induction: $P(s_0) \land T(s_0, s_1) \land \neg P(s_1)$

<img src="ind0.png" alt="define induction state space" style="width:55%;" align="center"/>


# Bonus: k-induction
- we can prove the example `Counter` correct by induction
- [original example on slide 99](https://zipcpu.com/tutorial/class-verilog.pdf )
- base case: $I(s_0) \land \neg P(s_0)$: `counter === 0 && !(counter =/= 500)`
- induction: $P(s_0) \land T(s_0, s_1) \land \neg P(s_1)$

<img src="ind3.png" alt="define induction state space" style="width:55%;" align="center"/>


# Bonus: k-induction
- we can prove the example `Counter` correct by induction
- [original example on slide 99](https://zipcpu.com/tutorial/class-verilog.pdf )
- base case: $I(s_0) \land P(s_0)$: `counter === 0 && !(counter =/= 500)`
- induction: $P(s_0) \land T(s_0, s_1) \land \neg P(s_1)$

<img src="ind1.png" alt="inductive invariant" style="width:55%;" align="center"/>


# Bonus: k-induction
- we can prove the example `Counter` correct by induction
- [original example on slide 99](https://zipcpu.com/tutorial/class-verilog.pdf )
- base case: $I(s_0) \land P(s_0)$: `counter === 0 && !(counter =/= 500)`
- induction: $P(s_0) \land T(s_0, s_1) \land \neg P(s_1)$

<img src="ind2.png" alt="inductive" style="width:55%;" align="center"/>


# Bonus: k-induction
- we can prove the example `Counter` correct by induction
- [original example on slide 99](https://zipcpu.com/tutorial/class-verilog.pdf )
- base case: $I(s_0) \land P(s_0)$: `counter === 0 && !(counter =/= 500)`

- induction: $P(s_0) \land T(s_0, s_1) \land \neg P(s_1)$

In [ ]:
// induction step w/o strengthening
val ind0 = solve(Seq(P(s0), T(s0, s1), not(P(s1))))
println(ind0)

# Bonus: k-induction
- just need a strengthening assertion that states that `count <= 22`
- induction: $P(s_0) \land T(s_0, s_1) \land \neg P(s_1)$
- we can use a strengething invariant (`inv`) to constrain the starting space

In [ ]:
// induction step w/ strengthening
def inv(s: State) = gte(lit(22), s.counter)
// combine original property with strengthening invariant
def P2(s: State) = and(inv(s), P(s))

solve(Seq(P2(s0), T(s0, s1), not(P2(s1))))
